In [ ]:
%pip install transformers
%pip install evaluate
%pip install accelerate -U
%pip install torch torchvision
%pip install rouge_score
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments,Seq2SeqTrainer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments
import evaluate
import csv
import random
import torch
from torch.utils.data import TensorDataset
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import numpy as np
import nltk
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
nltk.download('punkt')
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 895.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import pandas as pd
import torch
from transformers import T5TokenizerFast
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, input_max_length=1024, output_max_length=256):
        self.data = data
        self.tokenizer = tokenizer
        self.input_max_length = input_max_length
        self.output_max_length = output_max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        input_text = self.data.iloc[index]['input']
        target_text = self.data.iloc[index]['output']

        # Tokenize input and target text
        input_tokens = self.tokenizer.encode_plus(
            input_text,
            max_length=self.input_max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
            return_token_type_ids=False,
            return_attention_mask=True,
            return_overflowing_tokens=False,
            return_special_tokens_mask=False
        )
        target_tokens = self.tokenizer.encode(
            target_text,
            max_length=self.output_max_length,
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': input_tokens['input_ids'].flatten(),
            'attention_mask': input_tokens['attention_mask'].flatten(),
            'labels': target_tokens.flatten()
        }

# Load CSV data
data_df = pd.read_csv("/content/train_cv_data.csv")  # Change "your_data.csv" to your CSV file path

# Split data into train and evaluation sets
train_df, eval_df = train_test_split(data_df, test_size=0.2, random_state=42)

# Initialize T5 tokenizer
tokenizer = T5TokenizerFast.from_pretrained('t5-small')
tokenizer.add_special_tokens({'eos_token':'[EOS]'})

# Create datasets
train_dataset = CustomDataset(train_df, tokenizer)
eval_dataset = CustomDataset(eval_df, tokenizer)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=16, shuffle=False)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import T5ForConditionalGeneration
from transformers import T5Config, AutoConfig
# CONFIG_PATH = '/content/drive/MyDrive/Model_SparkLab/t5-small-finetuned/checkpoint-1500'
CONFIG_PATH = '/content/drive/MyDrive/Model_SparkLab/save_manually'
# config = T5Config(CONFIG_PATH)
# config = AutoConfig.from_pretrained(CONFIG_PATH)
model = T5ForConditionalGeneration.from_pretrained(CONFIG_PATH)
model.resize_token_embeddings(len(tokenizer))


Embedding(32101, 512)

In [ ]:
def remove_repeated_patterns(output_ids):
    # Convert the tensor to a list for easier manipulation
    output_list = output_ids.squeeze().tolist()

    def remove_repeats(lst):
        i = 0
        while i < len(lst):
            repeat_found = False
            # Look for repeating patterns starting from each position in the list
            for j in range(i + 1, len(lst)):
                if lst[i:j] == lst[j:j + (j - i)]:
                    # Remove the repeating sequence
                    lst = lst[:j] + lst[j + (j - i):]
                    repeat_found = True
                    break
            if not repeat_found:
                i += 1
        return lst

    # Remove repeating patterns from the output list
    cleaned_output_list = remove_repeats(output_list)

    # Convert the cleaned list back to a tensor
    cleaned_output_ids = torch.tensor(cleaned_output_list, device=output_ids.device).unsqueeze(0)
    return cleaned_output_ids

In [ ]:
model.eval()
# Perform inference on the first sample in the eval set
first_eval_sample = eval_dataset[15]
print(first_eval_sample)
# Prepare the input for inference
input_ids = first_eval_sample['input_ids'].unsqueeze(0).to(model.device) # Add batch dimension
attention_mask = first_eval_sample['attention_mask'].unsqueeze(0).to(model.device)  # Add batch dimension

# Generate output using the model
output_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask,
                            max_length=256)
cleaned_ids = remove_repeated_patterns(output_ids)
# print(output_ids)
# Decode the output tokens
output_text = tokenizer.decode(cleaned_ids[0], skip_special_tokens=True)

print("Generated Output:", output_text)

{'input_ids': tensor([7255,   19,  251,  ...,    7,   76,    1]), 'attention_mask': tensor([1, 1, 1,  ..., 1, 1, 1]), 'labels': tensor([14011,  8991,    32,  2754,   454,     5, 19844, 14011,   134,   440,
           51,  1208, 19844,    71,  2425,    11,  1906,   771,    28,     3,
            9,  4162,  1463,  1368,    16,   884,   313,     6,  3744, 19608,
            6,    11,   454,  4578, 20116,     3,  9886,     5,   749,  1926,
         1418,    12,   161, 13971,    11,    38,   294,    13,     3,     9,
          372,     6,    28,     3,     9,  1101,  1388,    12,  2736,    11,
            3,     9,  3148,    12,  1260,  1287,   313,     5, 14011,  5420,
         4267,  1433, 19844, 14011, 31963,     7,  5077,   179,     6,  7327,
         1387,     6,  9614,    18,  3082, 26744, 19844,     6,  1958,  5570,
            6,  1360,     3,    18,  6503,  1429, 10272,    15,    26,  3744,
        19608,   338,  4637,   382,    23,   889,  1429,  2200,    15,    26,
         2691,

In [ ]:
from tqdm import tqdm

In [ ]:
CONFIG_PATH = '/content/drive/MyDrive/Model_SparkLab/save_10epochs'
# config = T5Config(CONFIG_PATH)
# config = AutoConfig.from_pretrained(CONFIG_PATH)
model = T5ForConditionalGeneration.from_pretrained(CONFIG_PATH)
model.resize_token_embeddings(len(tokenizer))

Embedding(32101, 512)

In [ ]:
generated_texts = []
for i in tqdm(range(len(eval_dataset))):
  sample = eval_dataset[i]
  input_ids = sample['input_ids'].unsqueeze(0).to(model.device) # Add batch dimension
  attention_mask = sample['attention_mask'].unsqueeze(0).to(model.device)  # Add batch dimension

  # Generate output using the model
  output_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask,
                              max_length=256)
  cleaned_ids = remove_repeated_patterns(output_ids)
  # print(output_ids)
  # Decode the output tokens
  output_text = tokenizer.decode(cleaned_ids[0], skip_special_tokens=True)
  generated_texts.append(output_text)

100%|██████████| 140/140 [15:29<00:00,  6.64s/it]


In [ ]:
eval_df['pred_output'] = generated_texts


In [ ]:
from datasets import load_metric

metric = load_metric("rouge")
# Compute ROUGE scores
rouge_scores = [metric.compute(predictions=[pred], references=[ref], use_stemmer=True)
                for pred, ref in zip(eval_df['pred_output'], eval_df['output'])]

# Extract ROUGE scores and append them to the DataFrame
eval_df['rouge1_precision'] = [score['rouge1'].mid.precision for score in rouge_scores]
eval_df['rouge1_recall'] = [score['rouge1'].mid.recall for score in rouge_scores]
eval_df['rouge1_fmeasure'] = [score['rouge1'].mid.fmeasure for score in rouge_scores]

eval_df['rouge2_precision'] = [score['rouge2'].mid.precision for score in rouge_scores]
eval_df['rouge2_recall'] = [score['rouge2'].mid.recall for score in rouge_scores]
eval_df['rouge2_fmeasure'] = [score['rouge2'].mid.fmeasure for score in rouge_scores]

eval_df['rougeL_precision'] = [score['rougeL'].mid.precision for score in rouge_scores]
eval_df['rougeL_recall'] = [score['rougeL'].mid.recall for score in rouge_scores]
eval_df['rougeL_fmeasure'] = [score['rougeL'].mid.fmeasure for score in rouge_scores]


/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
eval_df.to_csv('results.csv', index=False)